# 1.2.2: Bikeshare (Implementing the State)

<br>



---



*Modeling and Simulation in Python*


Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---





We have now transformed a system into a model by defining the 4 types of variables.  Let's review our decisions:

<br>

Parameters:
* Total number of bikes: 12
* Length of simulation: 15 hours
* Time steps: 15 minutes (for a total of 60 time steps)
* Chance of bike being picked up in a time step: 50% (from Augie), 40% (from Moline)

<br>

Independent variables:
* Number of bikes starting at Augie: 0-12 (the initial state)

<br>

State variables:
* Number of bikes at Augie and Moline at a given moment

<br>

Metrics:
* Chance of having no empty racks?  Number of unhappy customers?

<br>

Now we need to begin the implementation process. We define the parameters and independent variables before the simulation begins, and metrics won't become important until we get a running simulation.  So our main aim will be to establish:
* a tool to keep track of the state variables as they change.
* a rule for how they will change (*a change function*).

Let's get to work!

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Simulatin.PNG width = 400>
</center>

## Keeping track of the state variables

In order to store the state, we'll use an *object* called a `Series`, which is part of a data analysis library called Pandas.  A couple definitions:

- An `object` is a catch-all term for a "thing" in python: a data storage vehicle of any sort.

- A `Series` is an object that holds a set of variable values in a table-like form, like a two-column spreadsheet.  We'll see an example of this in action in just a moment.


We'll start by importing the Pandas library (just as we have import Numpy in previous notebooks), and creating a `Series` called `bikeshare`.  This will be our *state object*: it will hold up-to-date information about the state of our system.

In [ ]:
import pandas as pd
state = dict(augie=10,moline=2)
bikeshare = pd.Series(state,name='Number of Bikes')
bikeshare.index.name = 'Location'
bikeshare

What did we just do?

* The first line imports the library, and gives it the nickname `pd`.  Importing the library makes all of its objects and methods available to the program, and the shorthand version slims down the code a bit.

<br>

* The second line creates a foundational Python object called a `dictionary`, which is a set of label-value pairs (think of it like a word and a definition--thus the name dictionary!).  The two labels in the dictionary ('Augie' and 'Moline') represent our *state variables*.  The *initial state* indicates that there are 10 bikes at Augie and 2 at Moline.

<br>

* The third line creates the `Series` and *assigns* it to a variable name `bikeshare`.  `bikeshare` now refers to the Series, in the same way that `g` refers to `9.8` when we write `g = 9.8`. A `Series` is a lot like a dictionary, but it is much easier to manipulate and is a more powerful way to store data.  The `name` is a *keyword argument* that describes the meaning of our values: '10' and '2' represent the number of bikes at the two locations.

<br>  

* The fourth line gives the *index* a name.  The index is the list of labels: the left column of the two-column table.  Here the index is the list of places with a bike rack.

<br>

* The final line sends the contents of `bikeshare` to be output, which is why the details of the Series appear when we run the cell.

We can read the variables inside a `Series` using the *dot operator*, like this:

In [ ]:
bikeshare.augie

In [ ]:
bikeshare.moline

Or, to display the state variables and their values, you can just enter the name of the object:

In [ ]:
bikeshare

We'll use Series a lot, for a range of purposes, so take a minute to understand this code.  Just remember that a Series is simply a two-column table, with an "index" of labels and a column of values.  Like this:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Series.PNG width = 300>
</center>

## Changing the state variables

Ok, we now have a place to store the state variables.  But in any useful model, the state is going to involve change.

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/state_of_confusion.png width = 400>
</center>

<br>

In a model, the process of change must be defined by a rule, or a set of rules, about how this change occurs.  We will call these rules a *change function*.  Much of this class will be spent exploring different types of change functions.  These rules might stay the same the whole time, or they might change, gradually or suddenly.  The rules might create random *stochastic* changes, or established *deterministic* changes.

<br>

The change occurs through *time steps*.  Each time step represents a certain amount of time, and each step represents a change in the state of the system.  We have defined our time step as being 15 minutes.

At its most basic, we enact the change function manually: that is, we can update the state by assigning new values to the variables.
For example, if a student moves a bike from Augie to Moline, we can figure out the new values and assign them:

In [ ]:
# First simple change function
bikeshare.augie = 9
bikeshare.moline = 3
bikeshare

That's not particularly efficient, though.  One step better, we can avoid doing the math ourselves by using *update operators*, `-=` and `+=`, to subtract 1 from
`augie` and add 1 to `moline`:

In [ ]:
# Second simple change function
bikeshare.augie -= 1
bikeshare.moline += 1
bikeshare

Try running the last cell again.  What happens to the state of the system?  Now run the previous code cell (with the title "first simple change function") again.  Finally, run the last cell again.

<br>

Each time you run the cell, it performs the action described by the code, even if you had already run that cell before.  This can be useful, but it can also cause trouble if you are not paying close attention!

## Summary and Exercises

This chapter introduces the tools we need to keep track of the state of a system and to change that state.   



---
<br>

🟨 🟨

### Exercise 1

Make a state Series, but this time add a third state variable in addition to `augie` and `moline`.  Call the third variable `rock_island`, with initial value 0, and display the state of the system.

In [ ]:
# Solution goes here


---
<br>

🟨 🟨

### Exercise 2

Create a new Series that contains the information shown in this two column table and output it:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Series_ex.PNG width = 300>
</center>

In [ ]:
# Solution goes here

---
<br>

🟨 🟨